In [1]:
# importing libraries
from time import time
from matplotlib.scale import LogScale
import pandas as pd 
import matplotlib.pyplot as plt
import math
import serial

ModuleNotFoundError: No module named 'matplotlib'

#Monitor Serial for UART Output

In [ ]:
from os import times_result

ser = serial.Serial(port='COM11', baudrate=115200,timeout = .02,parity = serial.PARITY_EVEN,rtscts = 1)
ser.close()
temp = list()
storage_SR = list()
storage_LONG = list()
isLONG = bool()

try:
    ser.open()
    try:
        while True:
            #read serial
            temp = ser.readline()

            #check if the line is the identifier string and set flags accordingly
            if temp == 'NEWMEAS_SR':
                storage_SR.clear()
                isLONG = 0
            elif temp == 'NEWMEAS_LONG':
                storage_LONG.clear()
                isLONG = 1
            elif temp == '':
                pass
            elif isLONG:
                storage_LONG.append(temp)
            elif not isLONG:
                storage_SR.append(temp)
                
            # create position and velocity lists 
            positions_SR = [0]*int(len(storage_SR)/2)
            velocities_SR =  [0]*int(len(storage_SR)/2)
            positions_LONG = [0]*int(len(storage_LONG)/2)
            velocities_LONG = [0]*int(len(storage_LONG)/2)
            
            # create time lists
            times_SR = [0]*int(len(storage_SR)/2)
            times_LONG = [0]*int(len(storage_LONG)/2)
            for i in positions_SR:
                times_SR[i] = .005*i
            for i in positions_LONG:
                times_LONG[i] = .005*i

            # create plots for step response plot
            if not isLONG:
                for i in storage_SR:
                    #sort the list into positions and velocities
                    if int.from_bytes(i,'big') % 2 == 1:
                        positions_SR[int.from_bytes(i,'big')/2 + .5] = storage_SR[int.from_bytes(i,'big')]
                    elif int.from_bytes(i,'big') % 2 == 0:
                        velocities_SR[int(int.from_bytes(i,'big')/2)] = storage_SR[int.from_bytes(i,'big')]

                    # Plot of pos vs time
                    plt.figure()
                    plt.plot(times_SR, positions_SR)
                    plt.xlabel('Time (s)')
                    plt.ylabel('Motor Position (rad)')
                    plt.title('Motor Position vs Time, Step Response')
                    plt.grid()

                    # Plot of vel vs time
                    plt.figure()
                    plt.plot(times_SR,velocities_SR)
                    plt.xlabel('Time (s)')
                    plt.ylabel('Velocity (rad/s)')
                    plt.title('Motor Velocity vs Time, Step Response')
                    plt.grid()
                
            if isLONG:
                for i in storage_SR:
                    #sort the list into positions and velocities
                    if int.from_bytes(i,'big') % 2 == 1:
                        positions_SR[int.from_bytes(i,'big')/2 + .5] = storage_SR(int.from_bytes(i,'big'))
                    else:
                        velocities_SR[int.from_bytes(i,'big')/2-1] = storage_SR(int.from_bytes(i,'big'))

                    # Plot of pos vs time
                    plt.figure()
                    plt.plot(times_LONG, positions_LONG)
                    plt.xlabel('Time (s)')
                    plt.ylabel('Motor Position (rad)')
                    plt.title('Motor Position vs Time, 30s Sample')
                    plt.grid()

                    # Plot of vel vs time
                    plt.figure()
                    plt.plot(times_LONG,velocities_LONG)
                    plt.xlabel('Time (s)')
                    plt.ylabel('Velocity (rad/s)')
                    plt.title('Motor Velocity vs Time, 30s Sample')
                    plt.grid()
    except:
        print('error in inner loop')
except:
    print('error in outer loop')

IndexError: list assignment index out of range